# BÀI TẬP NUMPY

## Đề Bài

Thực hiện các yêu cầu sau với dữ liệu 'vn_housing_dataset.csv':
1. Lấy ra thông tin từ các cột diện tích, chiều dài, chiều rộng, giá nhà.
2. Kiểm tra các thông tin bất thường, loại bỏ nếu thấy bất hợp lý.
3. Chuẩn hóa thông tin (ví dụ: giá nhà với đơn vị "tỷ" thì đổi hết sang đơn vị "triệu", nếu có).
4. Lấy thêm thông tin từ cột ngày, format lại thành "Ngày_Tháng_Năm"
5. Lưu lại dữ liệu đã làm sạch, chuẩn hóa, loại bỏ bất thường… vào file csv khác

## Script

### Using

In [1]:
import numpy as np
import pandas as pd

### Contract Fields

In [2]:
df = pd.read_csv('vn_housing_dataset.csv')

In [3]:
mod_cols = ['Diện tích', 'Dài', 'Rộng', 'Giá/m2']

### Contract Methods

In [4]:
#
def mod_split(input):
    return input.str.strip().str.split()

In [5]:
#
def try_parse_num(input):
    return pd.to_numeric(input.str[0].str.replace(',', '.'), errors='coerce')

In [6]:
#
def float_low(input):
    return input.str.replace(',', '.').str.lower()

In [7]:
#
def round_detail(input, unit):
    return input.str.split().str[0].astype(float).round(2).astype(str).apply(lambda val: f"{val} {unit}")

### Processing

In [8]:
df_mod = df[mod_cols]
df_mod = df_mod.rename(columns={'Giá/m2': 'Giá'})

In [9]:
area_mod = df_mod['Diện tích']
length_mod = df_mod['Dài']
width_mod = df_mod['Rộng']
price_mod = df_mod['Giá']

In [10]:
area_mod_split = mod_split(area_mod)
length_mod_split = mod_split(length_mod)
width_mod_split = mod_split(width_mod)
price_mod_split = mod_split(price_mod)

In [11]:
area_value = try_parse_num(area_mod_split)
length_value = try_parse_num(length_mod_split)
width_value = try_parse_num(width_mod_split)
price_value = try_parse_num(price_mod_split)

In [12]:
valid_data = (area_value.notna()
              & length_value.notna()
              & width_value.notna()
              & price_value.notna()
              & (area_value > 0)
              & (length_value > 0)
              & (width_value > 0)
              & (price_value > 0)
              & (area_mod_split.str.len() > 1)
              & (length_mod_split.str.len() > 1)
              & (width_mod_split.str.len() > 1)
              & (price_mod_split.str.len() > 1))
df_valid = df_mod.loc[valid_data]

In [13]:
invalid_data = ~valid_data
df_invalid = df_mod.loc[invalid_data]

In [14]:
area_valid = df_valid['Diện tích']
length_valid = df_valid['Dài']
width_valid = df_valid['Rộng']
price_valid = df_valid['Giá']

In [15]:
area_valid = round_detail(float_low(area_valid), 'm²')
length_valid = round_detail(float_low(length_valid), 'm')
width_valid = round_detail(float_low(width_valid), 'm')
price_valid = round_detail(float_low(price_valid).apply(lambda x: [str(float(x.split()[0]) * 1000) if x.split()[-1] == 'tỷ/m²' else x.split()[0], x.split()[-1]]).str.join(' '), 'triệu/m²')